# Chroma

Hướng dẫn này bao gồm cách sử dụng **Chroma Vector Store** với **LangChain**.

`Chroma` là một **open-source AI application database** (cơ sở dữ liệu ứng dụng AI mã nguồn mở).

Trong hướng dẫn này, sau khi học cách sử dụng `langchain-chroma`, chúng ta sẽ triển khai các ví dụ về một công cụ **Text Search** (tìm kiếm văn bản) đơn giản sử dụng `Chroma`.

![search-example](./assets/02-chroma-with-langchain-flow-search-example.png)

```bash
pip install langchain-chroma chromadb langchain-text-splitters langchain-huggingface
```

## What is Chroma?

![logo](./assets/02-chroma-with-langchain-chroma-logo.png)

`Chroma` là **open-source vector database** (cơ sở dữ liệu vector mã nguồn mở) được thiết kế cho các ứng dụng AI.

Nó chuyên về lưu trữ các vector chiều cao và thực hiện tìm kiếm tương tự nhanh chóng, khiến nó trở nên lý tưởng cho các tác vụ như **semantic search** (tìm kiếm ngữ nghĩa), **recommendation systems** (hệ thống đề xuất) và **multimodal search** (tìm kiếm đa phương thức).

Với **developer-friendly APIs** (API thân thiện với nhà phát triển) và tích hợp liền mạch với các framework như **LangChain**, `Chroma` là công cụ mạnh mẽ để xây dựng các giải pháp hướng AI có khả năng mở rộng.

Tính năng lớn nhất của `Chroma` là nó sử dụng nội bộ **Indexing ([HNSW](https://en.wikipedia.org/wiki/Hierarchical_navigable_small_world))** (lập chỉ mục) và **Embedding ([all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2))** (nhúng) khi lưu trữ dữ liệu.


## LangChain Chroma Basic

### Chọn Mô hình Nhúng (Embedding Model)

Chúng ta tải **Embedding Model** (mô hình nhúng) bằng `langchain_huggingface`.

Nếu bạn muốn sử dụng một mô hình khác, hãy sử dụng mô hình đó.


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "Alibaba-NLP/gte-base-en-v1.5"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs={"trust_remote_code": True}
)

/Users/ngohongthai/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


### Tạo VectorDB

**Library** (thư viện) được hỗ trợ bởi **LangChain** không có hàm `upsert` và thiếu tính đồng nhất giao diện với các **Vector DBs** (cơ sở dữ liệu vector) khác, vì vậy chúng tôi đã triển khai một lớp **Python** mới.

Đầu tiên, tải một lớp **Python** từ **utils/chroma/basic.py**.


In [2]:
from utils.chroma.basic import ChromaDB

vector_store = ChromaDB(embeddings=embeddings)

Create `ChromaDB` object.

- **Mode** : `persistent`

- **Persistent Path** : `data/chroma.sqlite` (Used `SQLite` DB)

- **collection** : `test`

- **hnsw:space** : `cosine`

In [3]:
configs = {
    "mode": "persistent",
    "persistent_path": "data/chroma_text",
    "collection": "test",
    "hnsw:space": "cosine",
}

vector_store.connect(**configs)

### Load Text Documents Data

Trong hướng dẫn này, chúng ta sẽ sử dụng tài liệu truyện cổ tích **Hoàng Tử Bé** (A Little Prince).

Để đưa dữ liệu này vào **Chroma**, chúng ta sẽ xử lý dữ liệu trước.

Trước hết, chúng ta sẽ tải file `data/the_little_prince.txt` đã được trích xuất chỉ phần văn bản của tài liệu truyện cổ tích.


In [4]:
# If your "OS" is "Windows", add 'encoding=utf-8' to the open function
with open("./data/the_little_prince.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

Second, chunking the text imported into the `RecursiveCharacterTextSplitter` .

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

split_docs = text_splitter.create_documents([raw_text])

for docs in split_docs[:2]:
    print(f"Content: {docs.page_content}\nMetadata: {docs.metadata}", end="\n\n")

Content: The Little Prince
Written By Antoine de Saiot-Exupery (1900〜1944)
Metadata: {}

Content: [ Antoine de Saiot-Exupery ]
Metadata: {}



In [ ]:
# Preprocessing document for **Chroma**.
pre_dosc = vector_store.preprocess_documents(
    documents=split_docs,
    source="The Little Prince",
    author="Antoine de Saint-Exupéry",
    chapter=True,
)

In [7]:
pre_dosc[:2]

[Document(metadata={'source': 'The Little Prince', 'author': 'Antoine de Saint-Exupéry', 'chapter': 1, 'id': '933ffdb9-8c67-4d75-8899-b00535b96ab8'}, page_content='- we are introduced to the narrator, a pilot, and his ideas about grown-ups'),
 Document(metadata={'source': 'The Little Prince', 'author': 'Antoine de Saint-Exupéry', 'chapter': 1, 'id': '1a178a15-5fd1-4860-9226-668042b71abc'}, page_content='Once when I was six years old I saw a magnificent picture in a book, called True Stories from')]

## Manage Store

This section introduces four basic functions.

- `add`

- `upsert(parallel)`

- `query`

- `delete`

### Add

Add the new **Documents** .

An error occurs if you have the same **ID** .

In [8]:
vector_store.add(pre_documents=pre_dosc[:2])

In [9]:
uids = list(vector_store.unique_ids)
uids

['933ffdb9-8c67-4d75-8899-b00535b96ab8',
 '1a178a15-5fd1-4860-9226-668042b71abc']

In [10]:
vector_store.chroma.get(ids=uids[0])

{'ids': ['933ffdb9-8c67-4d75-8899-b00535b96ab8'],
 'embeddings': None,
 'documents': ['- we are introduced to the narrator, a pilot, and his ideas about grown-ups'],
 'uris': None,
 'data': None,
 'metadatas': [{'author': 'Antoine de Saint-Exupéry',
   'chapter': 1,
   'source': 'The Little Prince'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

Error occurs when trying to `add` duplicate `ids` .

In [11]:
vector_store.add(pre_documents=pre_dosc[:2])

Add of existing embedding ID: 933ffdb9-8c67-4d75-8899-b00535b96ab8
Add of existing embedding ID: 1a178a15-5fd1-4860-9226-668042b71abc
Insert of existing embedding ID: 933ffdb9-8c67-4d75-8899-b00535b96ab8
Insert of existing embedding ID: 1a178a15-5fd1-4860-9226-668042b71abc


### Upsert(parallel)

`Upsert` will `Update` a document or `Add` a new document if the same `ID` exists.

In [12]:
tmp_ids = [docs.metadata["id"] for docs in pre_dosc[:2]]
vector_store.chroma.get(ids=tmp_ids)

{'ids': ['933ffdb9-8c67-4d75-8899-b00535b96ab8',
  '1a178a15-5fd1-4860-9226-668042b71abc'],
 'embeddings': None,
 'documents': ['- we are introduced to the narrator, a pilot, and his ideas about grown-ups',
  'Once when I was six years old I saw a magnificent picture in a book, called True Stories from'],
 'uris': None,
 'data': None,
 'metadatas': [{'author': 'Antoine de Saint-Exupéry',
   'chapter': 1,
   'source': 'The Little Prince'},
  {'author': 'Antoine de Saint-Exupéry',
   'chapter': 1,
   'source': 'The Little Prince'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [13]:
pre_dosc[0].page_content = "Changed Content"
pre_dosc[0]

Document(metadata={'source': 'The Little Prince', 'author': 'Antoine de Saint-Exupéry', 'chapter': 1, 'id': '933ffdb9-8c67-4d75-8899-b00535b96ab8'}, page_content='Changed Content')

In [14]:
vector_store.upsert_documents(
    documents=pre_dosc[:2],
)
tmp_ids = [docs.metadata["id"] for docs in pre_dosc[:2]]
vector_store.chroma.get(ids=tmp_ids)

{'ids': ['933ffdb9-8c67-4d75-8899-b00535b96ab8',
  '1a178a15-5fd1-4860-9226-668042b71abc'],
 'embeddings': None,
 'documents': ['Changed Content',
  'Once when I was six years old I saw a magnificent picture in a book, called True Stories from'],
 'uris': None,
 'data': None,
 'metadatas': [{'author': 'Antoine de Saint-Exupéry',
   'chapter': 1,
   'id': '933ffdb9-8c67-4d75-8899-b00535b96ab8',
   'source': 'The Little Prince'},
  {'author': 'Antoine de Saint-Exupéry',
   'chapter': 1,
   'id': '1a178a15-5fd1-4860-9226-668042b71abc',
   'source': 'The Little Prince'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [15]:
# parallel upsert
vector_store.upsert_documents_parallel(
    documents=pre_dosc,
    batch_size=32,
    max_workers=10,
)

## Query Vector Store

Có hai cách để **Query** (truy vấn) **LangChain Chroma Vector Store**.

-   **Directly** (Trực tiếp): Truy vấn vector store trực tiếp bằng các phương thức như `similarity_search` hoặc `similarity_search_with_score`.

-   **Turning into retriever** (Chuyển thành retriever): Chuyển đổi vector store thành một đối tượng **retriever**, có thể được sử dụng trong các pipeline hoặc chain của **LangChain**.


### Query

Phương thức này được tạo bằng cách bao bọc các phương thức của `langchain-chroma`.

**Tham số (Parameters)**

-   `query:str` - Văn bản truy vấn để tìm kiếm.

-   `k:int = DEFAULT_K` - Số lượng kết quả trả về. Mặc định là 4.

-   `filter: Dict[str, str] | None = None` - Lọc theo metadata. Mặc định là None.

-   `where_document: Dict[str, str] | None = None` - dict được sử dụng để lọc theo documents. Ví dụ: {$contains: {"text": "hello"}}.

-   `**kwargs:Any` : Các đối số từ khóa bổ sung để truyền cho truy vấn collection của Chroma.

**Trả về (Returns)**

-   `List[Document]` - Danh sách các documents tương tự nhất với văn bản truy vấn và khoảng cách ở dạng float cho mỗi document. Điểm thấp hơn thể hiện sự tương đồng cao hơn.


**Simple Search**

In [16]:
docs = vector_store.query(query="Prince", top_k=2)

for doc in docs:
    print("ID:", doc.metadata["id"])
    print("Chapter:", doc.metadata["chapter"])
    print("Page Content:", doc.page_content)
    print()

ID: cd3d8047-71bf-4202-bf94-f95b19986284
Chapter: 7
Page Content: prince disturbed my thoughts.

ID: 35960dc8-8cb4-467a-a994-990d69858db4
Chapter: 6
Page Content: Oh, little prince! Bit by bit I came to understand the secrets of your sad little life... For a



**Filtering Search**

In [17]:
docs = vector_store.query(query="Prince", top_k=2, filters={"chapter": 20})

for doc in docs:
    print("ID:", doc.metadata["id"])
    print("Chapter:", doc.metadata["chapter"])
    print("Page Content:", doc.page_content)
    print()

ID: 0540b8fc-84b6-4051-84b4-f2dbe0d58018
Chapter: 20
Page Content: snow, the little prince at last came upon a road. And all roads lead to the abodes of men.

ID: da84b43c-b9eb-440f-972d-799a9161269c
Chapter: 20
Page Content: extinct forever... that doesn‘t make me a very great prince..."



**Cosine Similarity Search**

In [18]:
# Cosine Similarity
results = vector_store.query(query="Prince", top_k=2, cs=True, filters={"chapter": 20})

for doc, score in results:
    print("ID:", doc.metadata["id"])
    print("Chapter:", doc.metadata["chapter"])
    print("Page Content:", doc.page_content)
    print(f"Similarity Score: {round(score,2)*100:.1f}%")
    print()

ID: 0540b8fc-84b6-4051-84b4-f2dbe0d58018
Chapter: 20
Page Content: snow, the little prince at last came upon a road. And all roads lead to the abodes of men.
Similarity Score: 60.0%

ID: da84b43c-b9eb-440f-972d-799a9161269c
Chapter: 20
Page Content: extinct forever... that doesn‘t make me a very great prince..."
Similarity Score: 54.0%



### as_retriever()

Phương thức `as_retriever()` chuyển đổi một đối tượng `VectorStore` thành một đối tượng `Retriever`.

Một `Retriever` là một giao diện được sử dụng trong `LangChain` để truy vấn một vector store và truy xuất các tài liệu liên quan.

**Tham số**

- `search_type:Optional[str]` - Xác định loại tìm kiếm mà `Retriever` sẽ thực hiện. Có thể là `similarity` (mặc định), `mmr`, hoặc `similarity_score_threshold`.

- `search_kwargs:Optional[Dict]` - Các đối số từ khóa được chuyển cho hàm tìm kiếm.

    Có thể bao gồm những thứ như:

    `k` : Số lượng tài liệu cần trả về (Mặc định: 4)

    `score_threshold` : Ngưỡng độ liên quan tối thiểu cho `similarity_score_threshold`

    `fetch_k` : Số lượng tài liệu được chuyển cho thuật toán `MMR` (Mặc định: 20)
        
    `lambda_mult` : Độ đa dạng của kết quả được trả về bởi `MMR`; 1 cho độ đa dạng tối thiểu và 0 cho độ đa dạng tối đa. (Mặc định: 0.5)

    `filter` : Lọc theo metadata của tài liệu

**Trả về**

- `VectorStoreRetriever` - Lớp `Retriever` cho `VectorStore`.

### invoke()

Gọi `retriever` để lấy các tài liệu liên quan.

Điểm truy cập chính cho các lệnh gọi `retriever` đồng bộ.

**Tham số**

- `input:str` - Chuỗi truy vấn.

- `config:RunnableConfig | None = None` - Cấu hình cho `retriever`. Mặc định là `None`.

- `**kwargs:Any` - Các đối số bổ sung được chuyển cho `retriever`.

**Trả về**

- `List[Document]` : Danh sách các tài liệu liên quan.


In [19]:
from langchain_chroma import Chroma

client = Chroma(
    collection_name="test",
    persist_directory="data/chroma_text",
    collection_metadata={"hnsw:space": "cosine"},
    embedding_function=embeddings,
)

In [20]:
retriever = client.as_retriever(search_type="similarity", search_kwargs={"k": 2})
docs = retriever.invoke("Prince", filter={"chapter": 5})

for doc in docs:
    print("ID:", doc.id)
    print("Chapter:", doc.metadata["chapter"])
    print("Page Content:", doc.page_content)
    print()

ID: 7a45b263-d531-48c4-aed3-094ebd874df3
Chapter: 5
Page Content: Indeed, as I learned, there were on the planet where the little prince lived-- as on all planets--

ID: 35fc3041-a366-4b3c-8cc5-a18939c363cb
Chapter: 5
Page Content: Now there were some terrible seeds on the planet that was the home of the little prince; and these



### Delete

`Delete` the Documents.

You can use with `filter` .

In [21]:
len(vector_store.unique_ids)

1317

In [22]:
len([docs for docs in pre_dosc if docs.metadata["chapter"] == 1])

43

In [23]:
vector_store.delete_by_filter(
    unique_ids=list(vector_store.unique_ids), filters={"chapter": 1}
)

Success Delete 43 Documents


In [24]:
len(vector_store.unique_ids)

1274

In [25]:
vector_store.delete_by_filter(unique_ids=list(vector_store.unique_ids))

Success Delete 1274 Documents


In [26]:
len(vector_store.unique_ids)

0

Remove a **Huggingface Cache** , `vector_store` , `embeddings` and `client` .

If you created a **vectordb** directory, please **remove** it at the end of this tutorial.

In [27]:
from huggingface_hub import scan_cache_dir

del embeddings
del vector_store
del client
scan = scan_cache_dir()
scan.delete_revisions()

DeleteCacheStrategy(expected_freed_size=0, blobs=frozenset(), refs=frozenset(), repos=frozenset(), snapshots=frozenset())

## Trình quản lý tài liệu (Document Manager)

Chúng tôi đã phát triển một giao diện giúp **CRUD** của **VectorDB** dễ sử dụng trong các hướng dẫn.

Các tính năng như sau:

- `upsert`: Chèn hoặc cập nhật các tài liệu trong cơ sở dữ liệu vector với metadata và embeddings tùy chọn.

- `upsert_parallel`: Xử lý việc chèn hoặc cập nhật hàng loạt song song để cải thiện hiệu suất.

- `search`: Tìm kiếm k tài liệu tương tự nhất bằng cách sử dụng **cosine similarity** (Trong hướng dẫn này, chúng tôi cố định điểm tương đồng là cosine similarity).

- `delete`: Xóa tài liệu theo ID hoặc lọc dựa trên metadata hoặc nội dung.

Mỗi hàm được kế thừa và phát triển cho từng vector DB.

Trong hướng dẫn này, nó được phát triển cho **Chroma**.


Load **Chroma Client** and **Embedding** .

In [28]:
import chromadb

client = chromadb.Client()  # in-memory

In [29]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "Alibaba-NLP/gte-base-en-v1.5"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs={"trust_remote_code": True}
)

Load `ChromaDocumentManager` .

In [30]:
from utils.chroma.crud import ChromaDocumentMangager

cdm = ChromaDocumentMangager(
    client=client,
    embedding=embeddings,
    name="chroma",
    metadata={"created_by": "pupba"},
)

Preprocessing for `Upsert` .

In [31]:
test_docs = pre_dosc[:50]

ids = [doc.metadata["id"] for doc in test_docs]
texts = [doc.page_content for doc in test_docs]
metadatas = [{k: v for k, v in doc.metadata.items() if k != "id"} for doc in test_docs]

### Upsert

Phương thức `upsert` được thiết kế để **chèn** hoặc **cập nhật** các tài liệu trong cơ sở dữ liệu vector.

Nó nhận các tham số sau:

- **texts**: Một tập hợp các văn bản tài liệu cần chèn hoặc cập nhật.

- **metadatas**: Metadata tùy chọn được liên kết với mỗi tài liệu.

- **ids**: Các định danh duy nhất tùy chọn cho mỗi tài liệu.

- ****kwargs**: Các đối số từ khóa bổ sung để linh hoạt.


In [32]:
cdm.upsert(texts=texts[:5], metadatas=metadatas[:5], ids=ids[:5])

In [33]:
cdm.collection.get()["ids"]

['933ffdb9-8c67-4d75-8899-b00535b96ab8',
 '1a178a15-5fd1-4860-9226-668042b71abc',
 '3f8900ec-328b-4618-b3e7-31a11ca83430',
 'a4259958-c82f-441e-a40a-f4926381f356',
 '3b3f9ad6-ed0a-447d-959a-f127a9c74ee2']

### Upsert-Parallel

Phương thức `upsert_parallel` là phiên bản tối ưu hóa của `upsert` xử lý các tài liệu song song.

Các tham số sau được thêm vào:

- **batch_size**: Số lượng tài liệu cần xử lý trong mỗi lô (mặc định: 32).

- **workers**: Số lượng worker song song cần sử dụng (mặc định: 10).


In [34]:
cdm.upsert_parallel(
    texts=texts,
    metadatas=metadatas,
    ids=ids,
)

In [35]:
len(cdm.collection.get()["ids"])

50

### Tìm kiếm (Search)

Phương thức `search` trả về một danh sách các đối tượng `Document`, là k tài liệu tương tự nhất với truy vấn.

- **query**: Một chuỗi đại diện cho truy vấn tìm kiếm.

- **k**: Một số nguyên chỉ định số lượng kết quả hàng đầu cần trả về (mặc định là 10).

- ****kwargs**: Các đối số từ khóa bổ sung để linh hoạt trong các tùy chọn tìm kiếm. Điều này có thể bao gồm các bộ lọc metadata (`where`, `where_document`).


In [36]:
results = cdm.search("prince", k=2)
results

[Document(metadata={'id': 'ae06fced-f049-4b0d-8664-2fe86f7d13bb', 'score': 0.52, 'author': 'Antoine de Saint-Exupéry', 'chapter': 2, 'source': 'The Little Prince'}, page_content='- the narrator crashes in the desert and makes the acquaintance of the little prince'),
 Document(metadata={'id': '8b7712f1-38bf-4f80-a604-b3758e9dbae6', 'score': 0.45, 'author': 'Antoine de Saint-Exupéry', 'chapter': 1, 'source': 'The Little Prince'}, page_content='I pondered deeply, then, over the adventures of the jungle. And after some work with a colored')]

In [37]:
results = cdm.search("prince", k=2, where={"chapter": 1})
results

[Document(metadata={'id': '8b7712f1-38bf-4f80-a604-b3758e9dbae6', 'score': 0.45, 'author': 'Antoine de Saint-Exupéry', 'chapter': 1, 'source': 'The Little Prince'}, page_content='I pondered deeply, then, over the adventures of the jungle. And after some work with a colored'),
 Document(metadata={'id': '5abd8ea6-725f-4734-b2fc-4a87e8bc2842', 'score': 0.41, 'author': 'Antoine de Saint-Exupéry', 'chapter': 1, 'source': 'The Little Prince'}, page_content='to be always and forever explaining things to them.')]

### Delete

The `delete` method removes documents from the vector database based on specified criteria.

- `ids` : A list of document IDs to be deleted. If None, all documents delete.

- `filters` : A dictionary specifying filtering criteria for deletion. This can include metadata filters( `where` , `where_document` ).

- `**kwargs` : Additional keyword arguments for custom deletion options.


In [38]:
len(cdm.collection.get()["ids"])

50

In [39]:
ids = cdm.collection.get()["ids"][:20]
cdm.delete(ids=ids)
len(cdm.collection.get()["ids"])

30

In [40]:
ids = cdm.collection.get(where={"chapter": 1})["ids"]
print("Chapter 1 documents counts:", len(ids))

Chapter 1 documents counts: 23
